In [84]:
import cv2
import numpy as np
import os

In [9]:
def rotate_image(image, angle):
    height, width = image.shape[:2]
    center = (width // 2, height // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height),  borderValue=(255, 255, 255))
    return rotated_image

In [10]:
def zoom_image(image, zoom_factor):
    height, width = image.shape[:2]
    new_height, new_width = int(height * zoom_factor), int(width * zoom_factor)
    zoomed_image = cv2.resize(image, (new_width, new_height))

    if zoom_factor < 1.0:
        top_pad = (height - new_height) // 2
        bottom_pad = height - new_height - top_pad
        left_pad = (width - new_width) // 2
        right_pad = width - new_width - left_pad
        zoomed_image = cv2.copyMakeBorder(zoomed_image, top_pad, bottom_pad, left_pad, right_pad, cv2.BORDER_CONSTANT, value=(255, 255, 255))
    else:
        zoomed_image = zoomed_image[(new_height - height) // 2 : (new_height + height) // 2,
                                    (new_width - width) // 2 : (new_width + width) // 2]

    return zoomed_image

In [128]:
def shear_image(image, shear_factor):
    height, width = image.shape[:2]
    shift_x = shear_factor * height / 2
    shear_matrix = np.array([[1, shear_factor, -shift_x],
                             [0, 1, 0]], dtype=np.float32)
    sheared_image = cv2.warpAffine(image, shear_matrix, (width, height), borderValue=(255, 255, 255))
    return sheared_image

In [129]:
def elastic_transform(image, alpha, sigma):
    random_state = np.random.RandomState(None)
    shape = image.shape
    shape_size = shape[:2]

    dx = random_state.rand(*shape_size) * 2 - 1
    dy = random_state.rand(*shape_size) * 2 - 1

    dx = cv2.GaussianBlur(dx, (0, 0), sigma) * alpha
    dy = cv2.GaussianBlur(dy, (0, 0), sigma) * alpha

    x, y = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]))
    map_x = np.float32(x + dx)
    map_y = np.float32(y + dy)

    elastic_image = np.zeros_like(image)
    for i in range(shape[2]):
        elastic_image[..., i] = cv2.remap(image[..., i], map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))

    return elastic_image

In [130]:
def invert_image(image):
    return 255 - image

def erode_image(image, kernel_size=(5, 5)):
    kernel = np.ones(kernel_size, np.uint8)
    inverted_image = invert_image(image)
    eroded_inverted_image = cv2.erode(inverted_image, kernel, iterations=1)
    eroded_image = invert_image(eroded_inverted_image)
    return eroded_image

def dilate_image(image, kernel_size=(5, 5)):
    kernel = np.ones(kernel_size, np.uint8)
    inverted_image = invert_image(image)
    dilated_inverted_image = cv2.dilate(inverted_image, kernel, iterations=1)
    dilated_image = invert_image(dilated_inverted_image)
    return dilated_image

In [131]:
# Exemple d'utilisation
image = cv2.imread('Arad_Ancient_Hebrew/Text_1/Alphabet/1_Alef/1.bmp')
zoomed_image = zoom_image(image, 0.8)
rotated_image = rotate_image(zoomed_image, 15)
cv2.imshow('Rotated Image', rotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [125]:
def generate_random_images(image, nb_images):
    random_images = []
    for _ in range(nb_images):
        transformed_image = image.copy()
        # Choose between erode or dilate
        if np.random.rand() < 0.5:
            kernel_size = (np.random.randint(1, 21), np.random.randint(1, 21))
            transformed_image = erode_image(transformed_image, kernel_size)
        else:
            kernel_size = (np.random.randint(1, 16), np.random.randint(1, 16))
            transformed_image = dilate_image(transformed_image, kernel_size)
                
        # Choose between rotate or shear
        if np.random.rand() < 0.5:
            angle = np.random.uniform(-20, 20)
            transformed_image = rotate_image(transformed_image, angle)
        else:
            shear_factor = np.random.uniform(-0.5, 0.5)
            transformed_image = shear_image(transformed_image, shear_factor)
            
        # Elastic transform 
        alpha = np.random.uniform(0, 60)
        sigma = 5
        transformed_image = elastic_transform(transformed_image, alpha, sigma)

        # Zoom
        zoom_factor = np.random.uniform(0.5, 1)
        transformed_image = zoom_image(transformed_image, zoom_factor)

        random_images.append(transformed_image)

    return random_images

In [78]:
def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

In [88]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [103]:
def save_images_from_folder(folder, images, first_number) :
    count = first_number
    for image in images : 
        cv2.imwrite(folder + '/' + str(count) + '.bmp', image)
        count +=1

In [126]:
nb_image = 5

def generate_hebrew_images(directory):
    images = []
    subdirectories = get_immediate_subdirectories(directory)
    for subdirectory in subdirectories :
        current_labels = get_immediate_subdirectories(directory+"/"+subdirectory+"/Alphabet")
        for current_label in current_labels :
            current_label_folder = directory+"/"+subdirectory+"/Alphabet"+"/"+current_label
            print(current_label_folder)
            retrieve_images = load_images_from_folder(current_label_folder)
            numbers_images_folder = len(retrieve_images)
            if numbers_images_folder != 0 :
                for image in retrieve_images :
                    images_generated = generate_random_images(image, nb_image)
                    save_images_from_folder(current_label_folder, images_generated, numbers_images_folder + 1)
                    numbers_images_folder += nb_image

In [127]:
generate_hebrew_images("Arad_Ancient_Hebrew")

Arad_Ancient_Hebrew/Text_1/Alphabet/10_Yod
Arad_Ancient_Hebrew/Text_1/Alphabet/11_Kaf
Arad_Ancient_Hebrew/Text_1/Alphabet/12_Lamed
Arad_Ancient_Hebrew/Text_1/Alphabet/13_Mem
Arad_Ancient_Hebrew/Text_1/Alphabet/14_Nun
Arad_Ancient_Hebrew/Text_1/Alphabet/15_Sameh
Arad_Ancient_Hebrew/Text_1/Alphabet/16_Ayin
Arad_Ancient_Hebrew/Text_1/Alphabet/17_Pe
Arad_Ancient_Hebrew/Text_1/Alphabet/18_Tsadi
Arad_Ancient_Hebrew/Text_1/Alphabet/19_Qof
Arad_Ancient_Hebrew/Text_1/Alphabet/1_Alef
Arad_Ancient_Hebrew/Text_1/Alphabet/20_Resh
Arad_Ancient_Hebrew/Text_1/Alphabet/21_Shin
Arad_Ancient_Hebrew/Text_1/Alphabet/22_Tav
Arad_Ancient_Hebrew/Text_1/Alphabet/2_Bet
Arad_Ancient_Hebrew/Text_1/Alphabet/3_Gimel
Arad_Ancient_Hebrew/Text_1/Alphabet/4_Dalet
Arad_Ancient_Hebrew/Text_1/Alphabet/5_Hey
Arad_Ancient_Hebrew/Text_1/Alphabet/6_Waw
Arad_Ancient_Hebrew/Text_1/Alphabet/7_Zain
Arad_Ancient_Hebrew/Text_1/Alphabet/8_Het
Arad_Ancient_Hebrew/Text_1/Alphabet/9_Tet
Arad_Ancient_Hebrew/Text_111/Alphabet/10_Yod
Arad